In [12]:
import os
from dotenv import load_dotenv

load_dotenv()

MODEL = "llama3"


In [13]:
from langchain_community.llms import Ollama

from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

model.invoke("Tell me a joke")

"Here's one:\n\nWhy couldn't the bicycle stand up by itself?\n\n(Wait for it...)\n\nBecause it was two-tired!\n\nHope that made you laugh!"

In [14]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 

In [15]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [16]:
chain = prompt | model | parser


In [18]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("OSS_Stack_2022_v8.8_2022-12-01.pdf")
pages = loader.load_and_split()
pages

[Document(page_content='Best Practices for Website\nHosting\nNeST-PRO-GEN.01 | Version 0.1\nStatus: Draft | April, 2018\nGovernment of India\nOpen Source Software Stack\n(Open source software stack for use in  e-Governanc e applications)\nNovember 2022\nOTG-TDOC-OSS-2022-1 | Version 8.8\nDisclaimer: Open Source Software and the logos included is / are copy right ofrespective Owners / Open Source Community\nNATIONAL INFORMATICS CENTRE\nMinistry of Electronics & Information Technology\nNew Delhi-110 003', metadata={'source': 'OSS_Stack_2022_v8.8_2022-12-01.pdf', 'page': 0}),
 Document(page_content='Amendment log\nVersion\nNo Release \nDateDescription  Section(s) \nModified By                           \nPrepared byReviewed by Approved By\n8.829th \nNovember \n2022OSS Stack 2022 for NIC Reviewed. New \nTools added after \nfeedback from NIC \nOfficers.OTG Team, NICSrinivasa Raghavan K, \nDDG and SIO - TNSCDG,NIC\n8.7.509th\nNovember \n2022OSS Stack 2022 for NIC Updated Tools  \nwith the la

In [19]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [20]:
retriever = vectorstore.as_retriever()
retriever.invoke("stack")

[Document(page_content='Database Replication SymmetricDS (3.14.x / 1.7.x)   \nDatabase Management SQLyog (Community Edition)(13.1.x)\nNewSQL YugabyteDB (2.14.1.x), Vitess(15.x)\nData Management ODK (Open Data Collect) (2022.3.x), Netlogo (6.3.x), Fluentd(1.15.x)\nCaching (in memory DB) redis (7.0.x), memcached(1.6.x)\nGIS Server Geo Server (2.21.x), \nGeotools(25.x)Geo Server (2.21.x), \nGeotools(25.x)`Map server \n(8.0.x/7.6.x), MBTILES \nCommunity \nmodule(1.x),GEOWEB\nCACHE(1.x), \nJDBCConfig- \nExtension to \nGeoServer,GeoServer \nPrinting Module \nextension \nGIS Desktop Quantum GIS (3.28.x / \n3.22.x), GRASS GIS \n(8.x/7.8.x), gvSIG (2.5.x)Quantum GIS (3.28.x / \n3.22.x), GRASS GIS \n(8.x/7.8.x), gvSIG (2.5.x)Quantum GIS (3.28.x /\n3.22.x), GRASS GIS \n(8.x/7.8.x), \ngvSIG(2.5.x) , \nTILEMILL map \nstudio(1.0.x)\nGIS Database PostGIS (3.1.x), neo4j CE (4.4.x)\nPortal/CMS Drupal (9.4.x/9.3.x/7.x),\nwordpress(6.0),\nzope(5.x)Liferay Community \nEdition (7.4.x),\nStrapi.io CE (4.4.x

In [21]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [22]:
questions = [
    "What is the purpose of the pdf?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: What is the purpose of the pdf?
Answer: The primary purpose of this PDF document appears to be a recommendation for an Open Source Software (OSS) stack for use in e-Government applications managed by the National Informatics Centre (NIC). The document provides information on the recommended OSS tools and their versions, along with guidelines for adoption and support services.



In [23]:
chain.batch([{"question": q} for q in questions])

['Based on the context, it appears that the purpose of this PDF document is to provide a comprehensive guide for using Open Source Software (OSS) in e-Governance applications, specifically within the National Informatics Centre (NIC). The document outlines the recommended OSS stack for NIC, including priority and supplementary tools, as well as guidelines for adopting and utilizing these tools.']

In [26]:
for s in chain.stream({"question": "What are the latest software for mobile native(IOS) ?"}):
    print(s, end="", flush=True)

The provided document does not mention anything about the latest software for mobile native (iOS). The document appears to be a guide on open-source software (OSS) stack for NIC and it focuses on tools and technologies used in developing e-Governance applications. It does not provide information on iOS or other mobile operating systems.

If you are looking for information on the latest iOS version, I can suggest checking Apple's official website or reputable technology news sources for that information.